In [226]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
%matplotlib inline

In [227]:
data = pd.read_csv("Aviva.csv")

In [228]:
data.columns

Index(['Creative Width', 'Creative Height', 'Ad Position', 'Exchange',
       'Exchange ID', 'Device Type', 'Advertiser Currency', 'Impressions',
       'Active View: Measurable Impressions',
       'Active View: Viewable Impressions', 'Clicks', 'Total Conversions',
       'Revenue (Adv Currency)'],
      dtype='object')

In [229]:
data = data.drop(['Exchange ID', 'Advertiser Currency'], axis = 1)

In [230]:
data.columns = ['CW', 'CH', 'AdPosition', 'Exchange',
       'Device', 'Impressions', 'mImpressions',
       'vImpressions', 'Clicks', "Conversions",
       'Spend']

In [231]:
data.CW = data.CW.apply(str)
data.CH = data.CH.apply(str)

In [232]:
data["Size"] = data.CW + "x" + data.CH
data["CTR"] = data.Clicks / data.Impressions
data["Viewability"] = data.vImpressions / data.mImpressions
data["CPM"] = data.Spend / data.Impressions * 1000
data["CPA"] = data.Spend / data.Conversions

In [233]:
data = data.drop(['CW', 'CH', 'mImpressions', 'vImpressions'], axis=1)

In [234]:
data.loc[data.CPA == np.inf, "CPA"] = np.median(data.CPA)

In [235]:
np.sum(data.Spend) / np.sum(data.Conversions)

2.4426547186400938

In [236]:
data.head()

,AdPosition,Exchange,Device,Impressions,Clicks,Conversions,Spend,Size,CTR,Viewability,CPM,CPA
0,Unknown,Admeta,Desktop,17,0,12,0.042347,160x600,0.000000,0.588235,2.491000,0.003529
1,Unknown,Rubicon,Desktop,3235,1,6,4.204572,160x600,0.000309,0.512065,1.299713,0.700762
2,ABOVE_THE_FOLD,AppNexus,Desktop,9493,7,10,23.783270,160x600,0.000737,0.632013,2.505348,2.378327
3,BELOW_THE_FOLD,AppNexus,Desktop,16709,0,7,44.777355,160x600,0.000000,0.226668,2.679835,6.396765
4,BELOW_THE_FOLD,BidSwitch,Desktop,3,0,0,0.005513,160x600,0.000000,0.333333,1.837667,6.355343


In [237]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [238]:
X = data.loc[:, ["AdPosition", "Size", "Viewability", "CPM"]].values
y = data.loc[:, "CTR"].values

In [239]:
labelencoder = LabelEncoder()

In [240]:
X[1,:]

array(['Unknown', '160x600', 0.5120651833281104, 1.2997131375579596], dtype=object)

In [241]:
X[:, 0] = labelencoder.fit_transform(X[:,0])
X[:, 1] = labelencoder.fit_transform(X[:,1])

In [242]:
X[1,:]

array([2, 0, 0.5120651833281104, 1.2997131375579596], dtype=object)

In [243]:
onehotencoder = OneHotEncoder(categorical_features=[0])

In [244]:
X = onehotencoder.fit_transform(X).toarray()

In [245]:
X[1,:]

array([ 0.        ,  0.        ,  1.        ,  0.        ,  0.51206518,
        1.29971314])

In [246]:
onehotencoder1 = OneHotEncoder(categorical_features=[3])

In [247]:
X = onehotencoder1.fit_transform(X).toarray()

In [248]:
X[1,:]

array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.51206518,  1.29971314])

In [249]:
from sklearn.cross_validation import train_test_split

In [250]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [251]:
import statsmodels.formula.api as sm

In [252]:
X = np.append(arr=np.ones((len(X[:,0]), 1)).astype(int), values = X, axis=1)


In [280]:
X_opt = X[:, [0, 3, 5]]

In [281]:
X[1, :]

array([ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.51206518,  1.29971314])

In [282]:
regressor_summary = sm.OLS(endog = y, exog = X_opt).fit()

In [283]:
print(regressor_summary.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     5.415
Date:                Wed, 19 Apr 2017   Prob (F-statistic):            0.00537
Time:                        17:35:56   Log-Likelihood:                 803.89
No. Observations:                 152   AIC:                            -1602.
Df Residuals:                     149   BIC:                            -1593.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5.257e-05      0.000      0.385      0.7